In [1]:
import pandas as pd

df = pd.read_parquet("/kaggle/input/unlp-2025-shared-task-span-identification/train.parquet")#'train.parquet')
cv = pd.read_csv("/kaggle/input/span-ident-cv-split-csv/cv_split.csv")#'cv_split.csv')
df = df.merge(cv, on='id', how='left')

df_test = pd.read_csv("/kaggle/input/unlp-2025-shared-task-span-identification/test.csv")#'test.csv')

In [2]:
import spacy

from spacy.training.iob_utils import biluo_to_iob, doc_to_biluo_tags
from tqdm.autonotebook import tqdm
tqdm.pandas()

df.trigger_words = df.trigger_words.apply(lambda x: [] if x is None else x)

In [3]:
PRETRAINED_MODEL = 'microsoft/mdeberta-v3-base'
TRAIN_LEN = 512
FULL_LEN = 1550

In [4]:
from transformers import AutoTokenizer
import pandas as pd
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
# from transformers import AutoTokenizer
# import pandas as pd
# from datasets import Dataset
# from tqdm.autonotebook import tqdm

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

# def convert_to_seq_labeling(text, tokenizer, trigger_spans=None, seq_len=TRAIN_LEN):
#     tokenized_output = tokenizer(
#         text, return_offsets_mapping=True, add_special_tokens=True, max_length=seq_len,
#         truncation=True, padding=False
#     )
#     tokens = tokenized_output["input_ids"]
#     offsets = tokenized_output["offset_mapping"]
#     token_strings = tokenizer.convert_ids_to_tokens(tokens)
    
#     labels = [0] * len(tokens)
#     if trigger_spans is not None:
#         for start, end in trigger_spans:
#             for i, (tok_start, tok_end) in enumerate(offsets):
#                 if tok_start == 0 and tok_end == 0:
#                     continue
#                 if tok_start < end and tok_end > start:
#                     labels[i] = 1
#     tokenized_output['labels'] = labels
#     return tokenized_output


# # df['seq_labels'] = df.progress_apply(lambda row: convert_to_seq_labeling(row['content'], tokenizer, row['trigger_words'], seq_len=FULL_LEN), axis=1)
# # for column in df.seq_labels.iloc[0].keys():
# #     df[column] = df.seq_labels.apply(lambda x: x.get(column))

# # Process training data
# tqdm.pandas()
# train_df = df[df.fold != 4].copy()
# train_df['seq_labels'] = train_df.progress_apply(lambda row: convert_to_seq_labeling(row['content'], tokenizer, row['trigger_words'], seq_len=TRAIN_LEN), axis=1)
# for column in train_df.seq_labels.iloc[0].keys():
#     train_df[column] = train_df.seq_labels.apply(lambda x: x.get(column))

# # Process validation data
# val_df = df[df.fold == 4].copy()
# val_df['seq_labels'] = val_df.progress_apply(lambda row: convert_to_seq_labeling(row['content'], tokenizer, row['trigger_words'], seq_len=FULL_LEN), axis=1)
# for column in val_df.seq_labels.iloc[0].keys():
#     val_df[column] = val_df.seq_labels.apply(lambda x: x.get(column))

# # Process test data
# df_test['seq_labels'] = df_test.progress_apply(lambda row: convert_to_seq_labeling(row['content'], tokenizer, None, seq_len=FULL_LEN), axis=1)
# for column in df_test.seq_labels.iloc[0].keys():
#     df_test[column] = df_test.seq_labels.apply(lambda x: x.get(column))

# # Create datasets
# columns = list(train_df.seq_labels.iloc[0].keys()) + ['content', 'trigger_words']
# ds_train = Dataset.from_pandas(train_df[columns].reset_index(drop=True))
# ds_valid = Dataset.from_pandas(val_df[columns].reset_index(drop=True))

# columns = list(df_test.seq_labels.iloc[0].keys()) + ['content']
# ds_test = Dataset.from_pandas(df_test[columns].reset_index(drop=True))


# import matplotlib.pyplot as plt
# import seaborn as sns

# # Calculate sequence lengths
# df['seq_len'] = df['input_ids'].apply(len)

# # Get statistics
# total_sequences = len(df)
# threshold = TRAIN_LEN
# long_sequences = (df['seq_len'] > threshold).sum()
# percentage = (long_sequences / total_sequences) * 100

# print(f"Sequences longer than {threshold}: {long_sequences} ({percentage:.2f}%)")
# print(f"Max sequence length: {df['seq_len'].max()}")
# print(f"Mean sequence length: {df['seq_len'].mean():.2f}")

# # Plot distribution
# plt.figure(figsize=(10, 5))
# sns.histplot(data=df['seq_len'], bins=50)
# plt.axvline(x=threshold, color='r', linestyle='--', label=f'Threshold ({threshold})')
# plt.title('Sequence Length Distribution')
# plt.xlabel('Sequence Length')
# plt.ylabel('Count')
# plt.legend()
# plt.show()

In [6]:
from transformers import AutoTokenizer
import pandas as pd
from datasets import Dataset
from tqdm.autonotebook import tqdm

def convert_to_seq_labeling(text, tokenizer, trigger_spans=None, chunk_size=512):
    # Tokenize entire text without truncation
    tokenized = tokenizer(
        text, 
        return_offsets_mapping=True, 
        add_special_tokens=False,
        truncation=False
    )
    
    chunks = []
    offsets = tokenized["offset_mapping"]
    tokens = tokenized["input_ids"]
    
    # Split into chunks (account for [CLS]/[SEP])
    for i in range(0, len(tokens), chunk_size - 2):
        chunk_tokens = tokens[i:i+chunk_size-2]
        chunk_offsets = offsets[i:i+chunk_size-2]
        
        # Add special tokens
        chunk_tokens = [tokenizer.cls_token_id] + chunk_tokens + [tokenizer.sep_token_id]
        chunk_offsets = [(0, 0)] + chunk_offsets + [(0, 0)]
        
        chunk_data = {
            "input_ids": chunk_tokens,
            "attention_mask": [1] * len(chunk_tokens),
            "offset_mapping": chunk_offsets
        }
        
        # Get actual character span from offset mapping
        chunk_start = chunk_offsets[1][0]  # First real token's start
        chunk_end = chunk_offsets[-2][1]   # Last real token's end
        
        labels = [0] * len(chunk_tokens)
        if trigger_spans is not None:
            for span_start, span_end in trigger_spans:
                # Check if span overlaps with this chunk's character range
                if not (span_end <= chunk_start or span_start >= chunk_end):
                    # Find overlap within this chunk
                    for idx, (tok_start, tok_end) in enumerate(chunk_offsets):
                        # Skip special tokens
                        if tok_start == 0 and tok_end == 0:
                            continue
                            
                        # Directly use original offsets from tokenizer
                        token_abs_start = tok_start
                        token_abs_end = tok_end
                        
                        # Check overlap with original span
                        if (token_abs_start < span_end and token_abs_end > span_start):
                            labels[idx] = 1

        chunk_data["labels"] = labels
        chunks.append(chunk_data)
    
    return chunks

def convert_val_to_seq_labeling(text, tokenizer, trigger_spans=None, seq_len=TRAIN_LEN):
    tokenized_output = tokenizer(
        text, return_offsets_mapping=True, add_special_tokens=True, max_length=seq_len,
        truncation=True, padding=False
    )
    tokens = tokenized_output["input_ids"]
    offsets = tokenized_output["offset_mapping"]
    token_strings = tokenizer.convert_ids_to_tokens(tokens)

    labels = [0] * len(tokens)
    if trigger_spans is not None:
        for start, end in trigger_spans:
            for i, (tok_start, tok_end) in enumerate(offsets):
                if tok_start == 0 and tok_end == 0:
                    continue
                if tok_start < end and tok_end > start:
                    labels[i] = 1
    tokenized_output['labels'] = labels
    return tokenized_output
    
def process_train_data(df, chunk_size):
    """Process training data with chunking"""
    tqdm.pandas()
    
    # Generate chunks for each row
    df["chunks"] = df.progress_apply(
        lambda row: convert_to_seq_labeling(
            row["content"],
            tokenizer,
            row.get("trigger_words"),
            chunk_size=chunk_size
        ),
        axis=1
    )
    
    # Explode chunks into separate rows
    df = df.explode("chunks").reset_index(drop=True)
    
    # Extract chunk components
    for key in ["input_ids", "attention_mask", "labels", "offset_mapping"]:
        df[key] = df["chunks"].apply(lambda x: x.get(key))
    
    # Add token_type_ids (if not present)
    if "token_type_ids" not in df.chunks.iloc[0]:
        df["token_type_ids"] = df["input_ids"].apply(lambda x: [0]*len(x))
    else:
        df["token_type_ids"] = df["chunks"].apply(lambda x: x.get("token_type_ids"))
    
    return df.drop(columns=["chunks"])


def process_val_test_data(df, seq_len):
    """Modified processing incorporating trigger span handling"""
    tqdm.pandas()
    
    df['seq_labels'] = df.progress_apply(
        lambda row: convert_val_to_seq_labeling(
            row['content'],
            tokenizer,
            row.get('trigger_words', None),  # Handle both validation and test cases
            seq_len=seq_len
        ),
        axis=1
    )
    
    # Extract all tokenizer outputs
    for column in df.seq_labels.iloc[0].keys():
        df[column] = df.seq_labels.apply(lambda x: x.get(column))
    
    return df


# Process training data with chunking
train_df_init = df[df.fold != 4].copy()
train_df_processed = process_train_data(train_df_init, chunk_size=TRAIN_LEN)

# Process validation data (original method)
val_df_init = df[df.fold == 4].copy()
val_df_processed = process_val_test_data(val_df_init, seq_len=FULL_LEN)

# Process test data (original method)
df_test_processed = process_val_test_data(df_test.copy(), seq_len=FULL_LEN)

# Create final datasets
train_columns = ['input_ids', 'attention_mask', 'token_type_ids', 'labels', 
                 'content', 'trigger_words', 'offset_mapping']
val_columns = ['input_ids', 'token_type_ids', 'attention_mask','offset_mapping', 'labels', 'content',  'trigger_words' ]
test_columns = ['input_ids', 'token_type_ids', 'attention_mask','offset_mapping', 'labels', 'content' ]

ds_train = Dataset.from_pandas(train_df_processed[train_columns].reset_index(drop=True))
ds_valid = Dataset.from_pandas(val_df_processed[val_columns].reset_index(drop=True))
ds_test = Dataset.from_pandas(df_test_processed[test_columns].reset_index(drop=True))


# to calculate positive class balance later
df['seq_labels'] = df.progress_apply(lambda row: convert_val_to_seq_labeling(row['content'], tokenizer, row['trigger_words'], seq_len=FULL_LEN), axis=1)
for column in df.seq_labels.iloc[0].keys():
    df[column] = df.seq_labels.apply(lambda x: x.get(column))

  0%|          | 0/3058 [00:00<?, ?it/s]

  0%|          | 0/764 [00:00<?, ?it/s]

  0%|          | 0/5735 [00:00<?, ?it/s]

  0%|          | 0/3822 [00:00<?, ?it/s]

In [7]:
def test_case():
    #text = "The Industrial Revolution began in Britain in the late 18th century and marked a major turning point in history."
    #trigger_spans = [[4, 25], [35, 42], [50, 67], [87, 100] ]
    text = 'Новий огляд мапи DeepState від російського військового експерта, кухара путіна 2 розряду, спеціаліста по снарядному голоду та ректора музичної академії міноборони рф Євгєнія Пригожина. \nПригожин прогнозує, що невдовзі настане день звільнення Криму і день розпаду росії. Каже, що передумови цього вже створені. \n*Відео взяли з каналу \nФД\n. \n@informnapalm'
    trigger_spans = [[27, 63], [65, 88], [90, 183], [186, 308]]
    
    # > 512 seq from val set
    #text = 'Российское руководство сделало террор инструментом внешней и внутренней политики.\nНачиная с 1994-го года РФ воюет с представителями исламского мира. Если брать советский период, то с 1979-го (с ввода войск в Афганистан).\nВ российских СМИ потерялись сообщения о ещё относительно недавней войне с Исламским Государством в Сирии. Медали за бомбардировки сирийцев розданы, герои назначены, а перманентные перестрелки на Северном Кавказе с регулярной гибелью российских военнослужащих не высвечиваются в СМИ.\nНо это не означает, что нет у Кремля недоброжелателей.\nОдин теракт в Москве высветил все пороки и болезни российского государства и вырожденческий инфантилизм российского общества в целом.\nНеспособность российского руководства быстро реагировать на новую угрозу, системное воровство, зависимость общества от нарративов СМИ, где во всём виновны украинцы, с одновременными требованиями от нас сочувствий, а также лояльное отношение к пыткам, ставшими нормой, которые применяются к подозреваемым уже практически в прямом эфире – это массовый психоз, ставший нормой.\nСистемно продуцируя ненависть и смерть, рано или поздно они приходят к самим авторам.\nКремль устраивает системный террор в Сирии и в Украине, убивая тысячи мирных граждан в пыточных, обстреливая снарядами и ракетами города, но при этом обвиняет всех вокруг в случившемся в Москве.\nГибель беззащитных гражданских, а особенно детей – это всегда ужасная трагедия, но российским гражданам, требующих большого сочувствия, стоит посмотреть правде в глаза и понять, что мировым рекордсменом по убийству ни в чем не повинных гражданских, включая маленьких детей, является их собственная страна. \nЧто-то мы не слышали сочувствия о терроре в Буче, Изюме, Мариуполе, Херсоне, когда россияне пытали у убивали людей тысячами, рассказывая по своим телеканалам про «инсценировки» и «зверства Азова».\nРеальность приходит вне зависимости от заговаривания её словами, и новый фактор в виде мести ИГ может быть таким же системным, как и российский террор.\n👉\n \nПолное видео.'
    #trigger_spans = [[0, 80], [1034, 1049], [1076, 1105], [1171, 1187], [1209, 1248], [1348, 1396], [1554, 1620], [1738, 1778], [1985, 2002]]
    for i in trigger_spans:
        print(text[i[0]:i[1]])
    
    chunk_size = 8 # 512

    # Tokenize and chunk
    chunks = convert_to_seq_labeling(text, tokenizer, trigger_spans, chunk_size)

    # Print results
    for i, chunk in enumerate(chunks):
        print(f"\nChunk {i+1}:")
        print(f"Tokens: {tokenizer.convert_ids_to_tokens(chunk['input_ids'])}")
        print(f"Offsets: {chunk['offset_mapping']}")
        print(f"Labels: {chunk['labels']}")

test_case()

від російського військового експерта
кухара путіна 2 розряду
спеціаліста по снарядному голоду та ректора музичної академії міноборони рф Євгєнія Пригожина
Пригожин прогнозує, що невдовзі настане день звільнення Криму і день розпаду росії. Каже, що передумови цього вже створені

Chunk 1:
Tokens: ['[CLS]', '▁Нови', 'й', '▁', 'огляд', '▁ма', 'пи', '[SEP]']
Offsets: [(0, 0), (0, 4), (4, 5), (5, 6), (6, 11), (11, 14), (14, 16), (0, 0)]
Labels: [0, 0, 0, 0, 0, 0, 0, 0]

Chunk 2:
Tokens: ['[CLS]', '▁Deep', 'State', '▁від', '▁', 'російськ', 'ого', '[SEP]']
Offsets: [(0, 0), (16, 21), (21, 26), (26, 30), (30, 31), (31, 39), (39, 42), (0, 0)]
Labels: [0, 0, 0, 1, 1, 1, 1, 0]

Chunk 3:
Tokens: ['[CLS]', '▁військов', 'ого', '▁експерт', 'а', ',', '▁кух', '[SEP]']
Offsets: [(0, 0), (42, 51), (51, 54), (54, 62), (62, 63), (63, 64), (64, 68), (0, 0)]
Labels: [0, 1, 1, 1, 1, 0, 1, 0]

Chunk 4:
Tokens: ['[CLS]', 'ара', '▁пут', 'іна', '▁2', '▁розр', 'яду', '[SEP]']
Offsets: [(0, 0), (68, 71), (71, 75), (

In [8]:
max_length_train = max(len(x) for x in ds_train['input_ids'])
max_length_val = max(len(x) for x in ds_valid['input_ids'])
max_length_test = max(len(x) for x in ds_test['input_ids'])

print(max_length_train)
print(max_length_val)
print(max_length_test)

512
1516
1445


In [9]:
from transformers import AdamW, get_linear_schedule_with_warmup


EPOCHS = 1

from transformers import AutoTokenizer, AutoModelForTokenClassification


model = AutoModelForTokenClassification.from_pretrained(
    PRETRAINED_MODEL,
    id2label={0: 0, 1: 1},
    label2id={0: 0, 1: 1},
)
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
import os
import random
import numpy as np
import torch

def set_seeds(seed):
    """Set seeds for reproducibility """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        

set_seeds(seed=42)

In [11]:
os.environ['WANDB_DISABLED'] = 'true'

import math
from transformers import Trainer, pipeline, TrainingArguments
from typing import Any
from transformers.trainer_utils import EvalPrediction


train_args = TrainingArguments(
    output_dir='model_checkpoints_mdebertav3_binary',
    logging_dir='./model_logs_mdebertav3_binary',
    learning_rate=2e-5,
    weight_decay=0.01,
    lr_scheduler_type='cosine',
    warmup_ratio=0.0,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    #bf16=True,
    # report_to="wandb",
    optim='adamw_torch',
    eval_strategy='steps',
    save_strategy="steps",
    eval_steps=100,
    logging_steps=10,
    save_steps=100,
    save_total_limit=10,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    load_best_model_at_end=True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
# import wandb

# # Initialize with team/entity
# wandb.init(
#     project="unlp-span-ident-task",
#     entity="bazdyrev99-igor-sikorsky-kyiv-polytechnic-institute", 
#     name='mdebertav3-binary'
# )

In [13]:
from itertools import chain

positive_class_balance = pd.Series(list(chain(*df.labels.tolist()))).mean()
positive_class_balance

0.2453814229012187

In [14]:
import math
from transformers import Trainer, pipeline, TrainingArguments
from typing import Any
from tqdm.autonotebook import tqdm
from transformers.trainer_utils import EvalPrediction

def extract_chars_from_spans(spans):
    """
    Given a list of spans (each a tuple (start, end)),
    return a set of character indices for all spans.
    """
    char_set = set()
    for start, end in spans:
        # Each span covers positions start, start+1, ..., end-1.
        char_set.update(range(start, end))
    return char_set

class SpanEvaluationTrainer(Trainer):
    def __init__(
        self,
        model: Any = None,
        args: TrainingArguments = None,
        data_collator: Any = None,
        train_dataset: Any = None,
        eval_dataset: Any = None,
        tokenizer: Any = None,
        desired_positive_ratio: float = 0.25,
        **kwargs,
    ):
        """
        Initialize the Trainer with our custom compute_metrics.
        """
        super().__init__(
            model=model,
            args=args,
            data_collator=data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=tokenizer,
            compute_metrics=self.compute_metrics,  # assign our custom compute_metrics
            **kwargs,
        )
        self.desired_positive_ratio = desired_positive_ratio

    def _calculate_inner_metric(self, gt_spans_all, pred_spans_all):
        total_true_chars = 0
        total_pred_chars = 0
        total_overlap_chars = 0
        for true_spans, pred_spans in zip(gt_spans_all, pred_spans_all):
            if isinstance(true_spans, str):
                try:
                    true_spans = eval(true_spans)
                except Exception:
                    true_spans = []
                    
            # Convert spans to sets of character indices.
            true_chars = extract_chars_from_spans(true_spans)
            pred_chars = extract_chars_from_spans(pred_spans)
            
            total_true_chars += len(true_chars)
            total_pred_chars += len(pred_chars)
            total_overlap_chars += len(true_chars.intersection(pred_chars))
            
            union_chars = true_chars.union(pred_chars)
            
        # Compute precision, recall, and F1.
        precision = total_overlap_chars / total_pred_chars if total_pred_chars > 0 else 0
        recall = total_overlap_chars / total_true_chars if total_true_chars > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        metrics = {
            "precision": precision,
            "recall": recall,
            "f1": f1
        }
        return metrics

    def _find_optimal_threshold(self, probabilities, labels):
        """Finds the threshold that achieves the desired positive class balance."""
        best_th = 0.5  # Default starting point
        best_diff = float("inf")
        optimal_th = best_th
        
        for thold in np.linspace(0.01, 0.99, num=100):
            predictions = (probabilities[:, :, 1] >= thold).astype(int)
            true_predictions = [
                [p for (p, l) in zip(prediction, label) if l != -100]
                for prediction, label in zip(predictions, labels)
            ]
            total_pos = sum([sum(row for row in prediction) for prediction in true_predictions])
            total = sum([len(prediction) for prediction in true_predictions])
            
            positive_ratio = total_pos / total if total > 0 else 0
            
            diff = abs(positive_ratio - self.desired_positive_ratio)
            if diff < best_diff:
                best_diff = diff
                optimal_th = thold
        
        return optimal_th
        
        
    def compute_metrics(self, eval_pred: EvalPrediction) -> dict:
        eval_dataset = self.eval_dataset
        logits, labels = eval_pred
        probabilities = torch.softmax(torch.tensor(logits), dim=-1).cpu().numpy()
    
        #thresholds = np.linspace(0.1, 0.5, num=41)
        thresholds = [self._find_optimal_threshold(probabilities, labels)]
        results = []
        best_f1 = -1
        best_th = 0
        best_metrics = None
    
        for thold in tqdm(thresholds):
            # Apply thresholding instead of argmax
            predictions = (probabilities[:, :, 1] >= thold).astype(int)
    
            true_predictions = [
                [p for (p, l) in zip(prediction, label) if l != -100]
                for prediction, label in zip(predictions, labels)
            ]
    
            pred_spans_all = []
            for pred, offsets in zip(true_predictions, eval_dataset['offset_mapping']):
                samplewise_spans = []
                current_span = None
                for token_label, span in zip(pred, offsets):
                    if token_label == 1:  # If the current token is labeled as an entity (1)
                        if current_span is None:
                            current_span = [span[0], span[1]]  # Start a new span
                        else:
                            current_span[1] = span[1]  # Extend the span to include the current token
                    else:  # If token_label == 0 (not an entity)
                        if current_span is not None:
                            samplewise_spans.append(tuple(current_span))  # Save completed span
                            current_span = None  # Reset for the next entity
    
                # If the last token was part of a span, save it
                if current_span is not None:
                    samplewise_spans.append(tuple(current_span))
    
                pred_spans_all.append(samplewise_spans)
    
            # Store results for this threshold
            current_metrics = self._calculate_inner_metric(eval_dataset['trigger_words'], pred_spans_all)
            if current_metrics['f1'] >= best_f1:
                best_f1 = current_metrics['f1']
                best_th = thold
                best_metrics = current_metrics
                best_metrics['thold'] = thold
                
            
            results.append(current_metrics)
        return best_metrics

In [15]:
torch.cuda.empty_cache()

In [16]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = SpanEvaluationTrainer(
    model=model,
    args=train_args,
    train_dataset=ds_train,
    eval_dataset=ds_valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
    desired_positive_ratio=positive_class_balance
)
trainer.train()

<ipython-input-14-1175221e7f1c>:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SpanEvaluationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


Step,Training Loss,Validation Loss,Precision,Recall,F1,Thold
100,0.403100,0.426032,0.596996,0.581819,0.589310,0.485152
200,0.383300,0.421938,0.603660,0.585552,0.594468,0.415859
300,0.418300,0.413985,0.609958,0.590771,0.600211,0.415859


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [17]:
FINETUNED_MODEL = '/kaggle/working/model_checkpoints_mdebertav3_binary/checkpoint-300'

model = AutoModelForTokenClassification.from_pretrained(
    FINETUNED_MODEL,
    id2label={0: 0, 1: 1},
    label2id={0: 0, 1: 1},
)
tokenizer = AutoTokenizer.from_pretrained(FINETUNED_MODEL)

In [18]:
trainer.model = model.cuda().eval()

In [19]:
valid_preds = trainer.predict(ds_valid)

  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
valid_preds.predictions.shape, valid_preds.label_ids.shape

((764, 1516, 2), (764, 1516))

In [21]:
trainer.compute_metrics((valid_preds.predictions, valid_preds.label_ids))

  0%|          | 0/1 [00:00<?, ?it/s]

{'precision': 0.6099580336256669,
 'recall': 0.5907710062284677,
 'f1': 0.600211220553053,
 'thold': 0.41585858585858587}

In [22]:
test_preds = trainer.predict(ds_test)

  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
test_probabilities = torch.softmax(torch.tensor(test_preds.predictions), dim=-1).cpu().numpy()

In [24]:
trainer._find_optimal_threshold(test_probabilities, test_preds.label_ids)

0.39606060606060606

In [26]:
# optimal th on (val set + test set) / 2
final_th = (0.4158+0.396)/2
final_th

0.40590000000000004

In [27]:
def inference_aggregation(probabilities, labels, offset_mappings, thold):
    predictions = (probabilities[:, :, 1] >= thold).astype(int)
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)
    ]
    pred_spans_all = []
    for pred, offsets in zip(true_predictions, offset_mappings):
        samplewise_spans = []
        current_span = None
        for token_label, span in zip(pred, offsets):
            if token_label == 1:  # If the current token is labeled as an entity (1)
                if current_span is None:
                    current_span = [span[0], span[1]]  # Start a new span
                else:
                    current_span[1] = span[1]  # Extend the span to include the current token
            else:  # If token_label == 0 (not an entity)
                if current_span is not None:
                    samplewise_spans.append(tuple(current_span))  # Save completed span
                    current_span = None  # Reset for the next entity
        
                    # If the last token was part of a span, save it
        if current_span is not None:
            samplewise_spans.append(tuple(current_span))
        
        pred_spans_all.append(samplewise_spans)
    return [str(row) for row in pred_spans_all]

In [28]:
valid_probabilities = torch.softmax(torch.tensor(valid_preds.predictions), dim=-1).cpu().numpy()
valid_results = inference_aggregation(valid_probabilities, valid_preds.label_ids, ds_valid['offset_mapping'], final_th)

In [29]:
import pandas as pd
import pandas.api.types
from sklearn.metrics import f1_score
import ast


class ParticipantVisibleError(Exception):
    """Custom exception for participant-visible errors."""
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    Compute span-level F1 score based on overlap.

    Parameters:
    - solution (pd.DataFrame): Ground truth DataFrame with row ID and token labels.
    - submission (pd.DataFrame): Submission DataFrame with row ID and token labels.
    - row_id_column_name (str): Column name for the row identifier.

    Returns:
    - float: The token-level weighted F1 score.

    Example:
    >>> solution = pd.DataFrame({
    ...     "id": [1, 2, 3],
    ...     "trigger_words": [[(612, 622), (725, 831)], [(300, 312)], []]
    ... })
    >>> submission = pd.DataFrame({
    ...     "id": [1, 2, 3],
    ...     "trigger_words": [[(612, 622), (700, 720)], [(300, 312)], [(100, 200)]]
    ... })
    >>> score(solution, submission, "id")
    0.16296296296296295
    """
    if not all(col in solution.columns for col in ["id", "trigger_words"]):
        raise ValueError("Solution DataFrame must contain 'id' and 'trigger_words' columns.")
    if not all(col in submission.columns for col in ["id", "trigger_words"]):
        raise ValueError("Submission DataFrame must contain 'id' and 'trigger_words' columns.")
    
    def safe_parse_spans(trigger_words):
        if isinstance(trigger_words, str):
            try:
                return ast.literal_eval(trigger_words)
            except (ValueError, SyntaxError):
                return []
        if isinstance(trigger_words, (list, tuple, np.ndarray)):
            return trigger_words
        return []

    def extract_tokens_from_spans(spans):
        tokens = set()
        for start, end in spans:
            tokens.update(range(start, end))
        return tokens
    
    solution = solution.copy()
    submission = submission.copy()

    solution["trigger_words"] = solution["trigger_words"].apply(safe_parse_spans)
    submission["trigger_words"] = submission["trigger_words"].apply(safe_parse_spans)

    # print(solution)
    # print()
    # print(submission)

    merged = pd.merge(
        solution,
        submission,
        on="id",
        suffixes=("_solution", "_submission")
    )

    total_true_tokens = 0
    total_pred_tokens = 0
    overlapping_tokens = 0

    for _, row in merged.iterrows():
        true_spans = row["trigger_words_solution"]
        pred_spans = row["trigger_words_submission"]
        # print(true_spans)
        # print()
        # print(pred_spans)
        # print()

        true_tokens = extract_tokens_from_spans(true_spans)
        pred_tokens = extract_tokens_from_spans(pred_spans)

        # print(true_tokens)
        # print()
        # print(pred_tokens)

        total_true_tokens += len(true_tokens)
        total_pred_tokens += len(pred_tokens)
        overlapping_tokens += len(true_tokens & pred_tokens)

    # print(true_tokens)
    # print()
    # print(pred_tokens)
    
    precision = overlapping_tokens / total_pred_tokens if total_pred_tokens > 0 else 0
    recall = overlapping_tokens / total_true_tokens if total_true_tokens > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return f1

In [30]:
from copy import deepcopy

df_gt = df[df.fold==4][['id', 'trigger_words']].reset_index(drop=True)
df_pred = deepcopy(df_gt)
df_pred['trigger_words'] = valid_results
score(df_gt, df_pred, row_id_column_name='id')

0.6035708682950821

In [ ]:
# test_results = inference_aggregation(test_probabilities, test_preds.label_ids, ds_test['offset_mapping'], final_th)

In [ ]:
# ss = pd.read_csv('sample_submission.csv')
# ss['trigger_words'] = test_results

In [ ]:
# ss.to_csv('submissions/mdebertav3-binary-cv0.605.csv', index=False)